In [ ]:
import cv2
from osgeo import gdal
import shapely
import database
import import_img
from imgview import ImgView

In [ ]:
db = database.Database('data/imagery.db')
A_path = 'data/DZB1211-500049L007001.tif'
B_path = 'data/D3C1208-200141A020_h.tif'

In [ ]:
fidA, iidA = import_img.import_img(db, A_path)
fidB, iidB = import_img.import_img(db, B_path)
dsA = gdal.Open(A_path, gdal.GA_ReadOnly)
dsB = gdal.Open(B_path, gdal.GA_ReadOnly)
ivA = ImgView(dsA.GetRasterBand(1))
ivB = ImgView(dsB.GetRasterBand(1))

In [ ]:
result_in_db = db.get_match(iidA, iidB)
H, area_B_in_A_wkt, n_points, last_update = result_in_db
area_B_in_A = shapely.from_wkt(area_B_in_A_wkt)

In [ ]:
box_B = shapely.box(0, 0, ivB.shape[1], ivB.shape[0])
area_B_in_A

In [ ]:
cursor=db.conn.cursor()
cursor.execute(
    "SELECT b_imgid, transform, ST_AsText(area_b_in_a), n_points, last_update "
    "FROM matchs "
    "WHERE a_imgid=?;",
    (iidA,)
)
poly_lst = []
for b_imgid, transform, area_b_in_a, n_points, last_update in cursor:
    poly_lst.append(shapely.from_wkt(area_b_in_a))
print(poly_lst)
shapely.multipolygons(poly_lst)